# Очистка и подготовка данных

## Работа с пропусками

Почему они бывают? Причин может быть много:

- данных просто нет (мы их не знаем);
- отсутствие данных имеют естественную причину и объяснимо;
- человеческая ошибка сбора/ввода;
- технические ошибки и проблемы, которые привели к потере данных.

Реальные данные почти всегда содержат пропуски. 
Нужно решить, что с ними делать исходя из причины их возникновения.  Неудачный выбор метода заполнения пропусков может не только не улучшить, но и сильно ухудшить результаты анализа.


In [1]:
import pandas as pd

In [9]:
titanic = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv', na_values='-') 
# titanic['Age'] = titanic['Age'].round().astype('Int64')

In [6]:
titanic['Age']

0        22
1        38
2        26
3        35
4        35
       ... 
886      27
887      19
888    <NA>
889      26
890      32
Name: Age, Length: 891, dtype: Int64

In [11]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    Int64  
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: Int64(1), float64(1), int64(5), object(5)
memory usage: 84.5+ KB


In [12]:
for col in titanic.columns:
    pct_missing = titanic[col].isna().mean()
    print(f'{col} - {pct_missing :.1%}')

PassengerId - 0.0%
Survived - 0.0%
Pclass - 0.0%
Name - 0.0%
Sex - 0.0%
Age - 19.9%
SibSp - 0.0%
Parch - 0.0%
Ticket - 0.0%
Fare - 0.0%
Cabin - 77.1%
Embarked - 0.2%


**Для самостоятельного изучения!**  
Для того чтобы понять, как правильно обработать пропуски, необходимо определить механизмы их формирования. 
Пропуски могут быть распределены в данных случайно, а могут – не случайно.
https://towardsdatascience.com/statistical-test-for-mcar-in-python-9fb617a76eac

### Игнорирование пропусков

In [13]:
# все методы pandas по-умолчанию просто не берут в расчет пропуски
print(titanic.Age.mean())
print(titanic.Age.median())
print(titanic.Age.mode()[0])
print(titanic.Age.std())
print(titanic.Age.var())

29.69327731092437
28.0
24
14.524527194574565
210.9618902259361


## Удаление строк с пропусками

Удаление строк, содержащих пропуски при их случайном распределении не приведет к существенному искажению свойств данных. Но если пропуски распределены не случайно, то смещение статистических свойств выборки могут быть значительными. В случаях, когда пропусков в данных много, это становится ощутимой проблемой и может сильно исказить результаты анализа.

Удаление столбцов с пропусками подходит только в том случае, если недостающие данные не являются информативными и пропусков чрезвычайно много (> 80%). 

In [14]:
# dropna удаляет все строки с пропущенными значениями
# titanic.dropna(inplace=True)
titanic.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    Int64  
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: Int64(1), float64(1), int64(5), object(5)
memory usage: 18.8+ KB


In [15]:
# посмотрите на сколько исказились статистики, если мы удалим все строки с пропусками
print(titanic.Age.mean())
print(titanic.Age.median())
print(titanic.Age.mode()[0])
print(titanic.Age.std())
print(titanic.Age.var())
print('------------------------------')
print(titanic.dropna().Age.mean())
print(titanic.dropna().Age.median())
print(titanic.dropna().Age.mode()[0])
print(titanic.dropna().Age.std())
print(titanic.dropna().Age.var())

29.69327731092437
28.0
24
14.524527194574565
210.9618902259361
------------------------------
35.67213114754098
36.0
36
15.6451600262331
244.77103224644208


In [16]:
# предположим, мы хотим удалить только те строки, в которых как минимум 11/12 значений заполнено
titanic.dropna(thresh=11).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  733 non-null    int64  
 1   Survived     733 non-null    int64  
 2   Pclass       733 non-null    int64  
 3   Name         733 non-null    object 
 4   Sex          733 non-null    object 
 5   Age          714 non-null    Int64  
 6   SibSp        733 non-null    int64  
 7   Parch        733 non-null    int64  
 8   Ticket       733 non-null    object 
 9   Fare         733 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     731 non-null    object 
dtypes: Int64(1), float64(1), int64(5), object(5)
memory usage: 75.2+ KB


### Удаление столбцов с пропусками

In [17]:
# удалять все столбцы с пропусками в данном случае – странно
titanic.dropna(axis = 1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
dtypes: float64(1), int64(5), object(3)
memory usage: 62.8+ KB


In [ ]:
# как и строки
titanic.dropna().info()

In [18]:
# у нас очень много пропусков в Cabin. Нам эта информация точно нунжа?
titanic.drop(['Cabin'], axis=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    Int64  
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: Int64(1), float64(1), int64(5), object(4)
memory usage: 77.6+ KB


### Замена пропусков

Замена на определенное значение

Замена пропущенных значений константой, которая заведомо не может попадать в реальные значения (-999, -1, “Нет информации” и пр.) позволит сгруппировать пропуски и рассматривать их как отдельную категорию.

Может быть полезно, когда не нужно делать никаких агрегированных расчетов и прогнозов, а наблюдения с пропусками есть.


In [19]:
# метод fillna заменить все NAN на указанное значение
titanic['Cabin'].fillna('no_info').isna().sum()

0

Замена средними

Для числовых признаков можно заполнить пропуски средним или медианным значением, полученное из остальных записей.
В случае категориальной дискретной характеристики наиболее часто используется заполнение модой. 

Это позволит нам не терять данные, но при этом может внести определенные искажения в выводы.

Лучше разумно подходить к заполнению пропусков и заполнять не просто средними, а средними по какой-то группе.    

In [20]:
fill_mean = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv') 
# возраст дискретен, при заполнении средними еще стоит округлить. Проигнорируем в учебных целях
fill_mean.Age.fillna(titanic['Age'].mean(), inplace=True)
fill_mean.Age.fillna(titanic['Age'].mean()).isna().sum()

0

In [21]:
print(titanic.Age.mean())
print(titanic.Age.median())
print(titanic.Age.mode()[0])
print(titanic.Age.std())
print(titanic.Age.var())
print('---------------------------')
print(fill_mean.Age.mean())
print(fill_mean.Age.median())
print(fill_mean.Age.mode()[0])
print(fill_mean.Age.std())
print(fill_mean.Age.var())

29.69327731092437
28.0
24
14.524527194574565
210.9618902259361
---------------------------
29.69795744560456
29.69327731092437
29.69327731092437
13.002015435047394
169.05240537321066


Замена медианой

Медиану можно считать (устойчивой) к наличию выбросов. Поэтому всегда стоит выбирать статистику, которая лучше описывает данные (например, среднее арифметическое под влиянием выбросов может сильно искажать представление о данных).

In [22]:
fill_median = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv') 
fill_median.Age.fillna(titanic.Age.median(), inplace=True)
fill_median.Age.fillna(titanic.Age.median()).isna().sum()

0

In [23]:
print(titanic.Age.mean())
print(titanic.Age.median())
print(titanic.Age.mode()[0])
print(titanic.Age.std())
print(titanic.Age.var())
print('--------------------------------')
print(fill_median.Age.mean())
print(fill_median.Age.median())
print(fill_median.Age.mode()[0])
print(fill_median.Age.std())
print(fill_median.Age.var())

29.69327731092437
28.0
24
14.524527194574565
210.9618902259361
--------------------------------
29.36158249158249
28.0
28.0
13.019696550973194
169.51249827942328


Замена модой

In [24]:
titanic.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [25]:
titanic_fill_mode = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv') 
titanic_fill_mode.Embarked.fillna(titanic.Embarked.mode()[0], inplace=True)

titanic_fill_mode.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

В большинстве случаев это слишком наивные варианты заполнения, как минимум стоит заполнять значениями в разрезе других признаков (когда применимо).

Заполнение пропусков с группировкой по другим столбцам

In [26]:
# мы не можем исключать, что средний возраст мужчин и женщин отличался
titanic.groupby('Sex').Age.median()

Sex
female    27
male      29
Name: Age, dtype: Int64

In [27]:
fill_median_by_gender = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv') 
fill_median_by_gender.Age.fillna(titanic.groupby('Sex').Age.transform('median'), inplace=True)
print(fill_median_by_gender.groupby('Sex').Age.median())

Sex
female    27.0
male      29.0
Name: Age, dtype: float64


In [28]:
# а может быть и в разных классах были пассажиры разного возраста?
titanic.groupby(['Sex', 'Pclass']).Age.median()

Sex     Pclass
female  1         35.0
        2         28.0
        3         21.5
male    1         40.0
        2         30.0
        3         25.0
Name: Age, dtype: float64

In [29]:
fill_median_by_groups = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/train.csv') 
fill_median_by_groups.Age.fillna(titanic.groupby(['Sex', 'Pclass']).Age.transform('median'), inplace=True)
fill_median_by_groups['Age'].isna().sum()

0

Заполнение следующими/предыдущими значениями

Данный метод применяется, как правило, при заполнении пропусков во временных рядах, когда последующие/предыдущие значения априори сильно взаимосвязаны с предыдущими.
При этом данный метод тоже может привести к существенным искажениям статистических свойств данных.

Возможна ситуация когда применение приведет к дублированию выбросов (заполнению пропусков аномальным значением). А если в данных много последовательно пропущенных значений, то использование метода также приводит к неправильным результатам.

In [30]:
city_day = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/city_day.csv')
city_day

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29526,Visakhapatnam,2020-06-27,15.02,50.94,7.68,25.06,19.54,12.47,0.47,8.55,23.30,2.24,12.07,0.73,41.0,Good
29527,Visakhapatnam,2020-06-28,24.38,74.09,3.42,26.06,16.53,11.99,0.52,12.72,30.14,0.74,2.21,0.38,70.0,Satisfactory
29528,Visakhapatnam,2020-06-29,22.91,65.73,3.45,29.53,18.33,10.71,0.48,8.42,30.96,0.01,0.01,0.00,68.0,Satisfactory
29529,Visakhapatnam,2020-06-30,16.64,49.97,4.05,29.26,18.80,10.03,0.52,9.84,28.30,0.00,0.00,0.00,54.0,Satisfactory


In [31]:
city_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [32]:
for col in city_day.columns:
    pct_missing = city_day[col].isna().mean()
    print(f'{col} - {pct_missing :.1%}')

City - 0.0%
Date - 0.0%
PM2.5 - 15.6%
PM10 - 37.7%
NO - 12.1%
NO2 - 12.1%
NOx - 14.2%
NH3 - 35.0%
CO - 7.0%
SO2 - 13.1%
O3 - 13.6%
Benzene - 19.0%
Toluene - 27.2%
Xylene - 61.3%
AQI - 15.9%
AQI_Bucket - 15.9%


In [33]:
city_day.fillna(method='ffill', inplace=True)

In [34]:
city_day.fillna(method='bfill', inplace=True)

In [35]:
for col in city_day.columns:
    pct_missing = city_day[col].isna().mean()
    print(f'{col} - {pct_missing :.1%}')

City - 0.0%
Date - 0.0%
PM2.5 - 0.0%
PM10 - 0.0%
NO - 0.0%
NO2 - 0.0%
NOx - 0.0%
NH3 - 0.0%
CO - 0.0%
SO2 - 0.0%
O3 - 0.0%
Benzene - 0.0%
Toluene - 0.0%
Xylene - 0.0%
AQI - 0.0%
AQI_Bucket - 0.0%


## Работа с датами

In [36]:
city_day = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/city_day.csv')
city_day['Date']

0        2015-01-01
1        2015-01-02
2        2015-01-03
3        2015-01-04
4        2015-01-05
            ...    
29526    2020-06-27
29527    2020-06-28
29528    2020-06-29
29529    2020-06-30
29530    2020-07-01
Name: Date, Length: 29531, dtype: object

In [37]:
# сразу распарсим даты в datetime формат
city_day = pd.read_csv('https://raw.githubusercontent.com/obulygin/SkillFactory/main/city_day.csv', parse_dates=['Date'])
city_day['Date']

0       2015-01-01
1       2015-01-02
2       2015-01-03
3       2015-01-04
4       2015-01-05
           ...    
29526   2020-06-27
29527   2020-06-28
29528   2020-06-29
29529   2020-06-30
29530   2020-07-01
Name: Date, Length: 29531, dtype: datetime64[ns]

In [38]:
# можем из даты извлекать любые составляющие 
city_day['Date'].dt.year

0        2015
1        2015
2        2015
3        2015
4        2015
         ... 
29526    2020
29527    2020
29528    2020
29529    2020
29530    2020
Name: Date, Length: 29531, dtype: int64

In [51]:
city_day['Date'].dt.month_name(locale='Russian')

0        Январь
1        Январь
2        Январь
3        Январь
4        Январь
          ...  
29526      Июнь
29527      Июнь
29528      Июнь
29529      Июнь
29530      Июль
Name: Date, Length: 29531, dtype: object

In [41]:
city_day['Date'].dt.day

0        3
1        4
2        5
3        6
4        0
        ..
29526    5
29527    6
29528    0
29529    1
29530    2
Name: Date, Length: 29531, dtype: int64

In [42]:
# корректно будет рассчитываться временная разница
city_day['Date'][0] - city_day['Date'][5]

Timedelta('-5 days +00:00:00')

In [43]:
# можно группировать даты по нужным интервалам
city_day.resample('M', on='Date').mean()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
Date,,,,,,,,,,,,,
2015-01-31,165.061143,276.971290,17.637473,25.180273,27.136729,40.768495,7.430047,30.952143,29.229066,4.946749,6.700585,2.017190,343.000000
2015-02-28,136.081321,234.939643,15.003558,26.781656,29.809372,47.917590,7.157173,23.989167,35.179080,5.021046,8.690575,3.733458,418.830189
2015-03-31,106.607368,135.044516,12.778162,25.131568,33.414213,22.132796,6.293241,18.444888,39.335243,12.683297,13.398599,2.765366,298.160920
2015-04-30,67.276307,187.991333,12.481006,24.089888,27.705263,22.508889,3.677488,18.021620,40.228555,3.063875,5.150952,2.277857,192.224852
2015-05-31,62.960269,271.050645,12.628011,25.799301,28.229816,29.854409,4.219861,15.328441,39.162989,3.945665,5.790773,2.597143,193.178378
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31,41.714778,86.625759,17.469883,20.332318,28.246295,20.743585,1.139529,13.298679,35.079557,13.393638,19.948017,2.909604,110.177267
2020-04-30,30.656291,68.308766,13.792563,14.707710,18.866055,18.012246,0.745440,11.178842,35.904359,2.530633,4.979677,1.167980,86.718056
2020-05-31,28.078289,69.423856,14.684585,14.427024,20.296750,15.114966,0.709377,10.314967,36.835376,3.473874,5.454352,2.539875,87.446292


In [53]:
city_day.resample('Q', on='Date').median()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
Date,,,,,,,,,,,,,
2015-03-31,109.780,198.105,10.285,23.010,24.330,23.310,2.24,8.505,32.860,2.110,4.860,1.350,323.0
2015-06-30,53.680,196.080,8.250,17.160,20.940,29.830,2.14,9.980,32.570,1.610,1.950,0.295,158.0
2015-09-30,40.985,136.880,8.120,14.125,19.245,30.435,1.40,8.560,17.120,0.330,2.020,0.150,146.0
2015-12-31,92.710,107.490,13.090,22.220,27.480,29.890,1.62,6.930,32.580,1.145,2.530,0.820,227.0
2016-03-31,83.440,132.905,10.050,21.680,17.950,24.600,1.25,6.580,37.230,0.935,2.005,0.460,194.0
2016-06-30,63.680,100.825,6.740,20.420,16.940,33.190,1.09,5.590,37.100,0.885,1.810,0.190,144.0
2016-09-30,36.470,76.970,9.460,18.470,18.395,27.890,1.02,5.630,21.690,0.665,2.090,0.445,110.0
2016-12-31,82.610,109.680,11.935,37.000,18.860,27.600,0.98,9.030,34.305,1.440,3.390,0.600,200.0
2017-03-31,76.860,117.070,10.430,33.640,14.680,21.930,0.60,7.780,36.210,0.200,1.670,0.120,176.0


## Объединение датафреймов

In [54]:
group_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'John', 'Robert', 'Alice', 'Sam'], 
        'last_name': ['Anderson', 'Jameson', 'Parker', 'Smith', 'Riddle']}
df_group_1 = pd.DataFrame(group_1)
df_group_1

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,John,Jameson
2,3,Robert,Parker
3,4,Alice,Smith
4,5,Sam,Riddle


In [55]:
group_2 = {
        'subject_id': ['10', '11', '12', '13', '14'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bruce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Bawner', 'Wayne', 'Smith']}
df_group_2 = pd.DataFrame(group_2)
df_group_2

,subject_id,first_name,last_name
0,10,Billy,Bonder
1,11,Brian,Black
2,12,Bran,Bawner
3,13,Bruce,Wayne
4,14,Betty,Smith


In [56]:
eval_data = {
        'subject_id': ['1', '2', '3', '4', '10', '11', '12', '14', '20', '21'],
        'score': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
scores = pd.DataFrame(eval_data)
scores

,subject_id,score
0,1,51
1,2,15
2,3,15
3,4,61
4,10,16
5,11,14
6,12,15
7,14,1
8,20,61
9,21,16


"Вертикальное" соединение датафреймов

In [57]:
all_groups = pd.concat([df_group_1, df_group_2])
all_groups

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,John,Jameson
2,3,Robert,Parker
3,4,Alice,Smith
4,5,Sam,Riddle
0,10,Billy,Bonder
1,11,Brian,Black
2,12,Bran,Bawner
3,13,Bruce,Wayne
4,14,Betty,Smith


"Горизонтальное" соединение датафреймов

In [58]:
all_groups_v = pd.concat([df_group_1, df_group_2], axis=1)
all_groups_v

,subject_id,first_name,last_name,subject_id,first_name,last_name
0,1,Alex,Anderson,10,Billy,Bonder
1,2,John,Jameson,11,Brian,Black
2,3,Robert,Parker,12,Bran,Bawner
3,4,Alice,Smith,13,Bruce,Wayne
4,5,Sam,Riddle,14,Betty,Smith


Объединение таблиц по столбцам

INNER join оставляет строчки, которые есть в обеих таблицах

In [59]:
# объединение происходит автоматически по столбцам с одинаковым названием
# по умолчанию используется inner join
pd.merge(all_groups, scores)

,subject_id,first_name,last_name,score
0,1,Alex,Anderson,51
1,2,John,Jameson,15
2,3,Robert,Parker,15
3,4,Alice,Smith,61
4,10,Billy,Bonder,16
5,11,Brian,Black,14
6,12,Bran,Bawner,15
7,14,Betty,Smith,1


LEFT join – каждой строчке в левой таблице ищет соответствие в правой (используется по-умолчанию)

In [60]:
pd.merge(all_groups, scores, how='left')

,subject_id,first_name,last_name,score
0,1,Alex,Anderson,51.0
1,2,John,Jameson,15.0
2,3,Robert,Parker,15.0
3,4,Alice,Smith,61.0
4,5,Sam,Riddle,NaN
5,10,Billy,Bonder,16.0
6,11,Brian,Black,14.0
7,12,Bran,Bawner,15.0
8,13,Bruce,Wayne,NaN
9,14,Betty,Smith,1.0


RIGHT join – каждой строчке в правой таблице ищет соответствие в левой

In [61]:
pd.merge(all_groups, scores, how='right')

,subject_id,first_name,last_name,score
0,1,Alex,Anderson,51
1,2,John,Jameson,15
2,3,Robert,Parker,15
3,4,Alice,Smith,61
4,10,Billy,Bonder,16
5,11,Brian,Black,14
6,12,Bran,Bawner,15
7,14,Betty,Smith,1
8,20,NaN,NaN,61
9,21,NaN,NaN,16


Outer join – оставляет все строчки

In [62]:
pd.merge(all_groups, scores, how='outer')

,subject_id,first_name,last_name,score
0,1,Alex,Anderson,51.0
1,2,John,Jameson,15.0
2,3,Robert,Parker,15.0
3,4,Alice,Smith,61.0
4,5,Sam,Riddle,NaN
5,10,Billy,Bonder,16.0
6,11,Brian,Black,14.0
7,12,Bran,Bawner,15.0
8,13,Bruce,Wayne,NaN
9,14,Betty,Smith,1.0


А что если названия столбцов не совпадают?

In [63]:
group_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'John', 'Robert', 'Alice', 'Sam'], 
        'last_name': ['Anderson', 'Jameson', 'Parker', 'Smith', 'Riddle']}
df_group_1 = pd.DataFrame(group_1)
group_2 = {
        'subject_id': ['1', '11', '12', '13', '14'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bruce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Bawner', 'Wayne', 'Smith']}
df_group_2 = pd.DataFrame(group_2)

eval_data = {
        'id': ['1', '2', '3', '4', '10', '11', '12', '14', '20', '21'],
        'score': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
scores = pd.DataFrame(eval_data)


In [64]:
pd.merge(all_groups, scores, left_on='subject_id', right_on='id')

,subject_id,first_name,last_name,id,score
0,1,Alex,Anderson,1,51
1,2,John,Jameson,2,15
2,3,Robert,Parker,3,15
3,4,Alice,Smith,4,61
4,10,Billy,Bonder,10,16
5,11,Brian,Black,11,14
6,12,Bran,Bawner,12,15
7,14,Betty,Smith,14,1


Абсолютно аналогично без проблем можно сделать объединение по нескольким столбцам.

Также объединение возможно при помощи функции `join`, но при помощи merge вы можете его сделать по любым индексам и столбцам, а при помощи `join` объединение происход строго по индексам (при этом `merge` медленее).

In [65]:
all_groups_i = all_groups.set_index('subject_id')
scores_i = scores.set_index('id')

display(all_groups_i)
display(scores_i)

,first_name,last_name
subject_id,,
1,Alex,Anderson
2,John,Jameson
3,Robert,Parker
4,Alice,Smith
5,Sam,Riddle
10,Billy,Bonder
11,Brian,Black
12,Bran,Bawner
13,Bruce,Wayne


,score
id,
1,51
2,15
3,15
4,61
10,16
11,14
12,15
14,1
20,61


In [66]:
pd.merge(all_groups, scores, left_index=True, right_index=True)

,subject_id,first_name,last_name,id,score
0,1,Alex,Anderson,1,51
0,10,Billy,Bonder,1,51
1,2,John,Jameson,2,15
1,11,Brian,Black,2,15
2,3,Robert,Parker,3,15
2,12,Bran,Bawner,3,15
3,4,Alice,Smith,4,61
3,13,Bruce,Wayne,4,61
4,5,Sam,Riddle,10,16
4,14,Betty,Smith,10,16


In [67]:
all_groups.join(scores)

,subject_id,first_name,last_name,id,score
0,1,Alex,Anderson,1,51
0,10,Billy,Bonder,1,51
1,2,John,Jameson,2,15
1,11,Brian,Black,2,15
2,3,Robert,Parker,3,15
2,12,Bran,Bawner,3,15
3,4,Alice,Smith,4,61
3,13,Bruce,Wayne,4,61
4,5,Sam,Riddle,10,16
4,14,Betty,Smith,10,16


In [68]:
# пример с одинаковыми именами разных столбцов

employees = pd.DataFrame({
    'yob':[1980, 1981, 1980, 1981, 1980, 1981, 1980, 1981],
    'name':['alice','alice','bob','bob','charlie','charlie','david','david'],
    'salary':[30000, 30000, 40000, 41000, 35000, 40000, 45000, 45000],
    'company_id':[1, 1, 2, 2, 1, 1, 2, 2]})

companies = pd.DataFrame({
    'id':[1, 1, 2, 2],
    'name':['company1', 'company2', 'company3', 'company4'],
    'revenue':[1130000, 1130000, 5000000, 500000]})

pd.merge(employees, companies, left_on='company_id', right_on='id')

,yob,name_x,salary,company_id,id,name_y,revenue
0,1980,alice,30000,1,1,company1,1130000
1,1980,alice,30000,1,1,company2,1130000
2,1981,alice,30000,1,1,company1,1130000
3,1981,alice,30000,1,1,company2,1130000
4,1980,charlie,35000,1,1,company1,1130000
5,1980,charlie,35000,1,1,company2,1130000
6,1981,charlie,40000,1,1,company1,1130000
7,1981,charlie,40000,1,1,company2,1130000
8,1980,bob,40000,2,2,company3,5000000
9,1980,bob,40000,2,2,company4,500000


In [70]:
pd.merge(employees, companies, left_on='company_id', right_on='id', suffixes=('_person', '_company'))

,yob,name_person,salary,company_id,id,name_company,revenue
0,1980,alice,30000,1,1,company1,1130000
1,1980,alice,30000,1,1,company2,1130000
2,1981,alice,30000,1,1,company1,1130000
3,1981,alice,30000,1,1,company2,1130000
4,1980,charlie,35000,1,1,company1,1130000
5,1980,charlie,35000,1,1,company2,1130000
6,1981,charlie,40000,1,1,company1,1130000
7,1981,charlie,40000,1,1,company2,1130000
8,1980,bob,40000,2,2,company3,5000000
9,1980,bob,40000,2,2,company4,500000


### Дубликаты

In [71]:
data = {"Name": ["James", "Alice", "Phil", "James"],
"Age": [24, 28, 40, 24],
"Sex": ["Male", "Female", "Male", "Male"]}

df = pd.DataFrame(data)
df

,Name,Age,Sex
0,James,24,Male
1,Alice,28,Female
2,Phil,40,Male
3,James,24,Male


In [80]:
df[df.duplicated(subset=['Sex', 'Age'])]

,Name,Age,Sex
3,James,24,Male


In [81]:
df = df.iloc[:,]

In [73]:
# можем удалять по наличию дубля в любом столбце/столбцах
df.drop_duplicates(subset = 'Sex')

,Name,Age,Sex
0,James,24,Male
1,Alice,28,Female


In [74]:
# можем выбирать, какой из дублей оставить
df.drop_duplicates(subset = 'Sex', keep = 'last')

,Name,Age,Sex
1,Alice,28,Female
3,James,24,Male


In [75]:
df.drop_duplicates(subset = 'Sex', keep = False)

,Name,Age,Sex
1,Alice,28,Female


### Спасибо за внимание! Буду рад ответить на ваши вопросы
Форма ОС: https://forms.gle/y8xaFwJqtbFSjUeG8